In [12]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
	# load the doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	return ' '.join(tokens)
 
# load all docs in a directory
def process_docs(directory, vocab):
	lines = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load and clean the doc
		line = doc_to_line(path, vocab)
		# add to list
		lines.append(line)
	return lines
 
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
# load all training reviews
positive_lines = process_docs('txt_sentoken/pos', vocab)
negative_lines = process_docs('txt_sentoken/neg', vocab)
# summarize what we have
print(len(positive_lines), len(negative_lines))

900 900


In [15]:
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

In [17]:
def process_docs(directory, vocab):
	lines = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load and clean the doc
		line = doc_to_line(path, vocab)
		# add to list
		lines.append(line)
	return lines
 
    
    # load all training reviews
positive_lines = process_docs('txt_sentoken/pos', vocab)
negative_lines = process_docs('txt_sentoken/neg', vocab)
# summarize what we have
print(len(positive_lines), len(negative_lines))

900 900


In [18]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
docs = positive_lines + negative_lines
tokenizer.fit_on_texts(docs)

In [19]:
# encode training data set
Xtrain = tokenizer.texts_to_matrix(docs, mode='freq')
print(Xtrain.shape)

(1800, 25768)


In [20]:
def process_docs(directory, vocab, is_trian):
	lines = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if is_trian and filename.startswith('cv9'):
			continue
		if not is_trian and not filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load and clean the doc
		line = doc_to_line(path, vocab)
		# add to list
		lines.append(line)
	return lines

In [21]:

# load all test reviews
positive_lines = process_docs('txt_sentoken/pos', vocab, False)
negative_lines = process_docs('txt_sentoken/neg', vocab, False)
docs = negative_lines + positive_lines
# encode training data set
Xtest = tokenizer.texts_to_matrix(docs, mode='freq')
print(Xtest.shape)

(200, 25768)


In [22]:
print(Xtrain.shape)

(1800, 25768)


In [23]:
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens
 
# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
	# load the doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	return ' '.join(tokens)
 
# load all docs in a directory
def process_docs(directory, vocab, is_trian):
	lines = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if is_trian and filename.startswith('cv9'):
			continue
		if not is_trian and not filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load and clean the doc
		line = doc_to_line(path, vocab)
		# add to list
		lines.append(line)
	return lines
 
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
# load all training reviews
positive_lines = process_docs('txt_sentoken/pos', vocab, True)
negative_lines = process_docs('txt_sentoken/neg', vocab, True)
# create the tokenizer
tokenizer = Tokenizer()

# fit the tokenizer on the documents
docs = negative_lines + positive_lines
#^This order over here and then in the ytrain line maps negative to 0 and positive to 1. 
#This will be important during prediction

tokenizer.fit_on_texts(docs)
# encode training data set
Xtrain = tokenizer.texts_to_matrix(docs, mode='freq')
ytrain = array([0 for _ in range(900)] + [1 for _ in range(900)])
 
# load all test reviews
positive_lines = process_docs('txt_sentoken/pos', vocab, False)
negative_lines = process_docs('txt_sentoken/neg', vocab, False)
docs = negative_lines + positive_lines
# encode training data set
Xtest = tokenizer.texts_to_matrix(docs, mode='freq')
ytest = array([0 for _ in range(100)] + [1 for _ in range(100)])
 
n_words = Xtest.shape[1]
# define network
model = Sequential()
model.add(Dense(50, input_shape=(n_words,), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=50, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Epoch 1/50
 - 1s - loss: 0.6918 - accuracy: 0.5294
Epoch 2/50
 - 1s - loss: 0.6827 - accuracy: 0.6739
Epoch 3/50
 - 1s - loss: 0.6640 - accuracy: 0.8622
Epoch 4/50
 - 1s - loss: 0.6331 - accuracy: 0.9156
Epoch 5/50
 - 1s - loss: 0.5934 - accuracy: 0.9261
Epoch 6/50
 - 1s - loss: 0.5472 - accuracy: 0.9417
Epoch 7/50
 - 1s - loss: 0.4999 - accuracy: 0.9489
Epoch 8/50
 - 1s - loss: 0.4520 - accuracy: 0.9550
Epoch 9/50
 - 1s - loss: 0.4078 - accuracy: 0.9539
Epoch 10/50
 - 1s - loss: 0.3648 - accuracy: 0.9622
Epoch 11/50
 - 1s - loss: 0.3276 - accuracy: 0.9728
Epoch 12/50
 - 1s - loss: 0.2931 - accuracy: 0.9783
Epoch 13/50
 - 1s - loss: 0.2627 - accuracy: 0.9800
Epoch 14/50
 - 1s - loss: 0.2355 - accuracy: 0.9844
Epoch 15/50
 - 1s - loss: 0.2118 - accuracy: 0.9894
Epoch 16/50
 - 1s - loss: 0.1905 - accuracy: 0.9906
Epoch 17/50
 - 1s - loss: 0.1714 - accuracy: 0.9928
Epoch 18/50
 - 1s - loss: 0.1547 - accuracy: 0.9939
Epoch 19/50
 - 1s - loss: 0.1405 - accuracy: 0.9939
Epoch 20/50
 - 1s - l

In [24]:
7
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from pandas import DataFrame
from matplotlib import pyplot
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens
 
# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
	# load the doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	return ' '.join(tokens)
 
# load all docs in a directory
def process_docs(directory, vocab, is_trian):
	lines = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if is_trian and filename.startswith('cv9'):
			continue
		if not is_trian and not filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load and clean the doc
		line = doc_to_line(path, vocab)
		# add to list
		lines.append(line)
	return lines
 
# evaluate a neural network model
def evaluate_mode(Xtrain, ytrain, Xtest, ytest):
	scores = list()
	n_repeats = 30
	n_words = Xtest.shape[1]
	for i in range(n_repeats):
		# define network
		model = Sequential()
		model.add(Dense(50, input_shape=(n_words,), activation='relu'))
		model.add(Dense(1, activation='sigmoid'))
		# compile network
		model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
		# fit network
		model.fit(Xtrain, ytrain, epochs=50, verbose=2)
		# evaluate
		loss, acc = model.evaluate(Xtest, ytest, verbose=0)
		scores.append(acc)
		print('%d accuracy: %s' % ((i+1), acc))
	return scores
 
# prepare bag of words encoding of docs
def prepare_data(train_docs, test_docs, mode):
	# create the tokenizer
	tokenizer = Tokenizer()
	# fit the tokenizer on the documents
	tokenizer.fit_on_texts(train_docs)
	# encode training data set
	Xtrain = tokenizer.texts_to_matrix(train_docs, mode=mode)
	# encode training data set
	Xtest = tokenizer.texts_to_matrix(test_docs, mode=mode)
	return Xtrain, Xtest
 
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
# load all training reviews
positive_lines = process_docs('txt_sentoken/pos', vocab, True)
negative_lines = process_docs('txt_sentoken/neg', vocab, True)
train_docs = negative_lines + positive_lines
# load all test reviews
positive_lines = process_docs('txt_sentoken/pos', vocab, False)
negative_lines = process_docs('txt_sentoken/neg', vocab, False)
test_docs = negative_lines + positive_lines
# prepare labels
ytrain = array([0 for _ in range(900)] + [1 for _ in range(900)])
ytest = array([0 for _ in range(100)] + [1 for _ in range(100)])
 
modes = ['binary', 'count', 'tfidf', 'freq']
results = DataFrame()
for mode in modes:
	# prepare data for mode
	Xtrain, Xtest = prepare_data(train_docs, test_docs, mode)
	# evaluate model on data for mode
	results[mode] = evaluate_mode(Xtrain, ytrain, Xtest, ytest)
# summarize results
print(results.describe())
# plot results
results.boxplot()
pyplot.show()

Epoch 1/50
 - 2s - loss: 0.4765 - accuracy: 0.7800
Epoch 2/50
 - 1s - loss: 0.0621 - accuracy: 0.9944
Epoch 3/50
 - 1s - loss: 0.0165 - accuracy: 1.0000
Epoch 4/50
 - 1s - loss: 0.0075 - accuracy: 1.0000
Epoch 5/50
 - 1s - loss: 0.0043 - accuracy: 1.0000
Epoch 6/50
 - 1s - loss: 0.0026 - accuracy: 1.0000
Epoch 7/50
 - 1s - loss: 0.0018 - accuracy: 1.0000
Epoch 8/50
 - 1s - loss: 0.0013 - accuracy: 1.0000
Epoch 9/50
 - 2s - loss: 9.9544e-04 - accuracy: 1.0000
Epoch 10/50
 - 1s - loss: 7.8771e-04 - accuracy: 1.0000
Epoch 11/50
 - 2s - loss: 6.4228e-04 - accuracy: 1.0000
Epoch 12/50
 - 1s - loss: 5.3790e-04 - accuracy: 1.0000
Epoch 13/50
 - 1s - loss: 4.5514e-04 - accuracy: 1.0000
Epoch 14/50
 - 1s - loss: 3.9111e-04 - accuracy: 1.0000
Epoch 15/50
 - 1s - loss: 3.4126e-04 - accuracy: 1.0000
Epoch 16/50
 - 1s - loss: 3.0005e-04 - accuracy: 1.0000
Epoch 17/50
 - 1s - loss: 2.6640e-04 - accuracy: 1.0000
Epoch 18/50
 - 1s - loss: 2.3784e-04 - accuracy: 1.0000
Epoch 19/50
 - 1s - loss: 2.1384e

Epoch 49/50
 - 1s - loss: 1.3703e-05 - accuracy: 1.0000
Epoch 50/50
 - 1s - loss: 1.3030e-05 - accuracy: 1.0000
3 accuracy: 0.925000011920929
Epoch 1/50
 - 2s - loss: 0.4775 - accuracy: 0.7861
Epoch 2/50
 - 1s - loss: 0.0691 - accuracy: 0.9933
Epoch 3/50
 - 1s - loss: 0.0205 - accuracy: 1.0000
Epoch 4/50
 - 1s - loss: 0.0083 - accuracy: 1.0000
Epoch 5/50
 - 1s - loss: 0.0041 - accuracy: 1.0000
Epoch 6/50
 - 1s - loss: 0.0023 - accuracy: 1.0000
Epoch 7/50
 - 1s - loss: 0.0014 - accuracy: 1.0000
Epoch 8/50
 - 1s - loss: 9.3557e-04 - accuracy: 1.0000
Epoch 9/50
 - 1s - loss: 6.6967e-04 - accuracy: 1.0000
Epoch 10/50
 - 1s - loss: 5.0439e-04 - accuracy: 1.0000
Epoch 11/50
 - 1s - loss: 3.9504e-04 - accuracy: 1.0000
Epoch 12/50
 - 1s - loss: 3.1677e-04 - accuracy: 1.0000
Epoch 13/50
 - 1s - loss: 2.5973e-04 - accuracy: 1.0000
Epoch 14/50
 - 1s - loss: 2.1696e-04 - accuracy: 1.0000
Epoch 15/50
 - 1s - loss: 1.8409e-04 - accuracy: 1.0000
Epoch 16/50
 - 1s - loss: 1.5783e-04 - accuracy: 1.0000

Epoch 46/50
 - 1s - loss: 1.5772e-05 - accuracy: 1.0000
Epoch 47/50
 - 1s - loss: 1.4798e-05 - accuracy: 1.0000
Epoch 48/50
 - 1s - loss: 1.3920e-05 - accuracy: 1.0000
Epoch 49/50
 - 1s - loss: 1.3094e-05 - accuracy: 1.0000
Epoch 50/50
 - 1s - loss: 1.2321e-05 - accuracy: 1.0000
6 accuracy: 0.9150000214576721
Epoch 1/50
 - 1s - loss: 0.5138 - accuracy: 0.7578
Epoch 2/50
 - 1s - loss: 0.0943 - accuracy: 0.9861
Epoch 3/50
 - 1s - loss: 0.0241 - accuracy: 1.0000
Epoch 4/50
 - 1s - loss: 0.0094 - accuracy: 1.0000
Epoch 5/50
 - 1s - loss: 0.0049 - accuracy: 1.0000
Epoch 6/50
 - 1s - loss: 0.0028 - accuracy: 1.0000
Epoch 7/50
 - 1s - loss: 0.0017 - accuracy: 1.0000
Epoch 8/50
 - 1s - loss: 0.0011 - accuracy: 1.0000
Epoch 9/50
 - 1s - loss: 7.7316e-04 - accuracy: 1.0000
Epoch 10/50
 - 1s - loss: 5.6976e-04 - accuracy: 1.0000
Epoch 11/50
 - 1s - loss: 4.4050e-04 - accuracy: 1.0000
Epoch 12/50
 - 1s - loss: 3.5268e-04 - accuracy: 1.0000
Epoch 13/50
 - 1s - loss: 2.9007e-04 - accuracy: 1.0000
Ep

Epoch 43/50
 - 1s - loss: 1.5590e-05 - accuracy: 1.0000
Epoch 44/50
 - 1s - loss: 1.4291e-05 - accuracy: 1.0000
Epoch 45/50
 - 1s - loss: 1.3132e-05 - accuracy: 1.0000
Epoch 46/50
 - 1s - loss: 1.2141e-05 - accuracy: 1.0000
Epoch 47/50
 - 1s - loss: 1.1161e-05 - accuracy: 1.0000
Epoch 48/50
 - 1s - loss: 1.0362e-05 - accuracy: 1.0000
Epoch 49/50
 - 1s - loss: 9.5999e-06 - accuracy: 1.0000
Epoch 50/50
 - 1s - loss: 8.9178e-06 - accuracy: 1.0000
9 accuracy: 0.9200000166893005
Epoch 1/50
 - 1s - loss: 0.4803 - accuracy: 0.7744
Epoch 2/50
 - 1s - loss: 0.0553 - accuracy: 0.9961
Epoch 3/50
 - 1s - loss: 0.0162 - accuracy: 1.0000
Epoch 4/50
 - 1s - loss: 0.0073 - accuracy: 1.0000
Epoch 5/50
 - 1s - loss: 0.0042 - accuracy: 1.0000
Epoch 6/50
 - 1s - loss: 0.0026 - accuracy: 1.0000
Epoch 7/50
 - 1s - loss: 0.0018 - accuracy: 1.0000
Epoch 8/50
 - 1s - loss: 0.0013 - accuracy: 1.0000
Epoch 9/50
 - 1s - loss: 0.0010 - accuracy: 1.0000
Epoch 10/50
 - 1s - loss: 8.2339e-04 - accuracy: 1.0000
Epoch 

Epoch 40/50
 - 1s - loss: 1.5619e-05 - accuracy: 1.0000
Epoch 41/50
 - 1s - loss: 1.4689e-05 - accuracy: 1.0000
Epoch 42/50
 - 1s - loss: 1.3814e-05 - accuracy: 1.0000
Epoch 43/50
 - 1s - loss: 1.3024e-05 - accuracy: 1.0000
Epoch 44/50
 - 1s - loss: 1.2305e-05 - accuracy: 1.0000
Epoch 45/50
 - 1s - loss: 1.1594e-05 - accuracy: 1.0000
Epoch 46/50
 - 1s - loss: 1.0958e-05 - accuracy: 1.0000
Epoch 47/50
 - 1s - loss: 1.0368e-05 - accuracy: 1.0000
Epoch 48/50
 - 1s - loss: 9.8422e-06 - accuracy: 1.0000
Epoch 49/50
 - 1s - loss: 9.3094e-06 - accuracy: 1.0000
Epoch 50/50
 - 1s - loss: 8.8345e-06 - accuracy: 1.0000
12 accuracy: 0.9200000166893005
Epoch 1/50
 - 1s - loss: 0.4655 - accuracy: 0.7856
Epoch 2/50
 - 1s - loss: 0.0563 - accuracy: 0.9939
Epoch 3/50
 - 1s - loss: 0.0157 - accuracy: 1.0000
Epoch 4/50
 - 1s - loss: 0.0074 - accuracy: 1.0000
Epoch 5/50
 - 1s - loss: 0.0040 - accuracy: 1.0000
Epoch 6/50
 - 1s - loss: 0.0023 - accuracy: 1.0000
Epoch 7/50
 - 1s - loss: 0.0015 - accuracy: 1.

...

 - 1s - loss: 0.0244 - accuracy: 1.0000
Epoch 50/50
 - 1s - loss: 0.0231 - accuracy: 1.0000
20 accuracy: 0.9100000262260437
Epoch 1/50
 - 1s - loss: 0.6926 - accuracy: 0.5100
Epoch 2/50
 - 1s - loss: 0.6881 - accuracy: 0.6883
Epoch 3/50
 - 1s - loss: 0.6754 - accuracy: 0.8861
Epoch 4/50
 - 1s - loss: 0.6531 - accuracy: 0.9172
Epoch 5/50
 - 1s - loss: 0.6224 - accuracy: 0.9217
Epoch 6/50
 - 1s - loss: 0.5841 - accuracy: 0.9383
Epoch 7/50
 - 1s - loss: 0.5426 - accuracy: 0.9367
Epoch 8/50
 - 1s - loss: 0.5003 - accuracy: 0.9467
Epoch 9/50
 - 1s - loss: 0.4570 - accuracy: 0.9528
Epoch 10/50
 - 1s - loss: 0.4152 - accuracy: 0.9600
Epoch 11/50
 - 1s - loss: 0.3768 - accuracy: 0.9622
Epoch 12/50
 - 1s - loss: 0.3421 - accuracy: 0.9694
Epoch 13/50
 - 1s - loss: 0.3110 - accuracy: 0.9733
Epoch 14/50
 - 1s - loss: 0.2818 - accuracy: 0.9800
Epoch 15/50
 - 1s - loss: 0.2552 - accuracy: 0.9806
Epoch 16/50
 - 1s - loss: 0.2315 - accuracy: 0.9861
Epoch 17/50
 - 1s - loss: 0.2107 - accuracy: 0.9883
E

Epoch 5/50
 - 1s - loss: 0.6024 - accuracy: 0.9328
Epoch 6/50
 - 1s - loss: 0.5596 - accuracy: 0.9400
Epoch 7/50
 - 1s - loss: 0.5136 - accuracy: 0.9472
Epoch 8/50
 - 1s - loss: 0.4684 - accuracy: 0.9528
Epoch 9/50
 - 1s - loss: 0.4251 - accuracy: 0.9617
Epoch 10/50
 - 1s - loss: 0.3846 - accuracy: 0.9633
Epoch 11/50
 - 1s - loss: 0.3460 - accuracy: 0.9694
Epoch 12/50
 - 1s - loss: 0.3122 - accuracy: 0.9700
Epoch 13/50
 - 1s - loss: 0.2813 - accuracy: 0.9800
Epoch 14/50
 - 1s - loss: 0.2541 - accuracy: 0.9839
Epoch 15/50
 - 1s - loss: 0.2293 - accuracy: 0.9867
Epoch 16/50
 - 1s - loss: 0.2074 - accuracy: 0.9894
Epoch 17/50
 - 1s - loss: 0.1878 - accuracy: 0.9917
Epoch 18/50
 - 1s - loss: 0.1705 - accuracy: 0.9928
Epoch 19/50
 - 1s - loss: 0.1546 - accuracy: 0.9944
Epoch 20/50
 - 1s - loss: 0.1404 - accuracy: 0.9956
Epoch 21/50
 - 1s - loss: 0.1277 - accuracy: 0.9961
Epoch 22/50
 - 1s - loss: 0.1166 - accuracy: 0.9967
Epoch 23/50
 - 1s - loss: 0.1064 - accuracy: 0.9978
Epoch 24/50
 - 1s

Epoch 12/50
 - 1s - loss: 0.3068 - accuracy: 0.9761
Epoch 13/50
 - 1s - loss: 0.2762 - accuracy: 0.9800
Epoch 14/50
 - 1s - loss: 0.2482 - accuracy: 0.9844
Epoch 15/50
 - 1s - loss: 0.2237 - accuracy: 0.9883
Epoch 16/50
 - 1s - loss: 0.2015 - accuracy: 0.9883
Epoch 17/50
 - 1s - loss: 0.1822 - accuracy: 0.9906
Epoch 18/50
 - 1s - loss: 0.1647 - accuracy: 0.9928
Epoch 19/50
 - 1s - loss: 0.1493 - accuracy: 0.9944
Epoch 20/50
 - 1s - loss: 0.1355 - accuracy: 0.9956
Epoch 21/50
 - 1s - loss: 0.1229 - accuracy: 0.9956
Epoch 22/50
 - 1s - loss: 0.1120 - accuracy: 0.9972
Epoch 23/50
 - 1s - loss: 0.1021 - accuracy: 0.9978
Epoch 24/50
 - 1s - loss: 0.0931 - accuracy: 0.9978
Epoch 25/50
 - 1s - loss: 0.0853 - accuracy: 0.9983
Epoch 26/50
 - 1s - loss: 0.0785 - accuracy: 0.9983
Epoch 27/50
 - 1s - loss: 0.0717 - accuracy: 1.0000
Epoch 28/50
 - 1s - loss: 0.0658 - accuracy: 1.0000
Epoch 29/50
 - 1s - loss: 0.0607 - accuracy: 1.0000
Epoch 30/50
 - 1s - loss: 0.0558 - accuracy: 1.0000
Epoch 31/50


 - 1s - loss: 0.1604 - accuracy: 0.9928
Epoch 19/50
 - 1s - loss: 0.1456 - accuracy: 0.9944
Epoch 20/50
 - 1s - loss: 0.1317 - accuracy: 0.9956
Epoch 21/50
 - 1s - loss: 0.1203 - accuracy: 0.9972
Epoch 22/50
 - 1s - loss: 0.1097 - accuracy: 0.9972
Epoch 23/50
 - 1s - loss: 0.0996 - accuracy: 0.9983
Epoch 24/50
 - 1s - loss: 0.0909 - accuracy: 0.9983
Epoch 25/50
 - 1s - loss: 0.0833 - accuracy: 0.9989
Epoch 26/50
 - 1s - loss: 0.0762 - accuracy: 1.0000
Epoch 27/50
 - 1s - loss: 0.0699 - accuracy: 1.0000
Epoch 28/50
 - 1s - loss: 0.0643 - accuracy: 1.0000
Epoch 29/50
 - 1s - loss: 0.0591 - accuracy: 1.0000
Epoch 30/50
 - 1s - loss: 0.0545 - accuracy: 1.0000
Epoch 31/50
 - 1s - loss: 0.0505 - accuracy: 1.0000
Epoch 32/50
 - 1s - loss: 0.0464 - accuracy: 1.0000
Epoch 33/50
 - 1s - loss: 0.0431 - accuracy: 1.0000
Epoch 34/50
 - 1s - loss: 0.0401 - accuracy: 1.0000
Epoch 35/50
 - 1s - loss: 0.0371 - accuracy: 1.0000
Epoch 36/50
 - 1s - loss: 0.0344 - accuracy: 1.0000
Epoch 37/50
 - 1s - loss

<Figure size 640x480 with 1 Axes>

In [55]:
# test positive text
text = 'Best movie ever!'
print(predict_sentiment(text, vocab, tokenizer, model))
# test negative text
text = 'This is a bad movie.'
print(predict_sentiment(text, vocab, tokenizer, model))

1.0
0.0


1/1 [==============================] - 0s 1ms/step
[[0.98006564]]
1.0
1/1 [==============================] - 0s 1ms/step
[[1.5453779e-07]]
0.0


In [66]:
t='Panipat is bad awful movie.'
t2='Border is good movie.'
tokens = clean_doc(t)
print(tokens)
# filter by vocab
tokens = [w for w in tokens if w in vocab]
print(tokens)
# convert to line
line = ' '.join(tokens)
print('Tokenized from Vocab: '+ line)
# encode
encoded = tokenizer.texts_to_matrix([line], mode='freq')
print(encoded)
# prediction


['Panipat', 'bad', 'awful', 'movie']
['bad', 'awful', 'movie']
Tokenized from Vocab: bad awful movie
[[0. 0. 0. ... 0. 0. 0.]]


In [61]:
# yhat = model.predict(encoded, verbose=0)
# print("Class: {}".format(yhat))
print(model.predict(encoded, verbose=0))

# yproba = model.predict_proba(encoded, verbose=0)    
# print("probability: {}" .format(yproba))
print(model.predict_proba(encoded, verbose=0))

model.predict(encoded, verbose=0)
model.predict_proba(encoded, verbose=0)    
print(predict_sentiment(t, vocab, tokenizer, model))
print(predict_sentiment(t2, vocab, tokenizer, model))


[[3.1213265e-11]]
[[3.1213265e-11]]


In [75]:
model.predict(encoded, verbose=0)==model.predict_proba(encoded, verbose=0)



0.0
1.0


'/Users/vivek/My Folder/Learning/imdb_sentiment_project/review_polarity'

In [80]:
import pickle

# saving model
filename = 'finalized_model.sav'
pickle.dump(model, open(os.getcwd()+ '/'+filename, 'wb'))

# saving tokenizer
with open(os.getcwd()+'/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)    

In [79]:
os.getcwd()

'/Users/vivek/My Folder/Learning/imdb_sentiment_project/review_polarity'

In [13]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens
 
# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
	# load the doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	return ' '.join(tokens)
 
# load all docs in a directory
def process_docs(directory, vocab, is_trian):
	lines = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if is_trian and filename.startswith('cv9'):
			continue
		if not is_trian and not filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load and clean the doc
		line = doc_to_line(path, vocab)
		# add to list
		lines.append(line)
	return lines
 
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
 
# load all training reviews
positive_lines = process_docs('txt_sentoken/pos', vocab, True)
negative_lines = process_docs('txt_sentoken/neg', vocab, True)
 
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
docs = negative_lines + positive_lines
tokenizer.fit_on_texts(docs)
 
# encode training data set
Xtrain = tokenizer.texts_to_matrix(docs, mode='freq')
print(Xtrain.shape)
 
# load all test reviews
positive_lines = process_docs('txt_sentoken/pos', vocab, False)
negative_lines = process_docs('txt_sentoken/neg', vocab, False)
docs = negative_lines + positive_lines
# encode training data set
Xtest = tokenizer.texts_to_matrix(docs, mode='freq')
print(Xtest.shape)

(1800, 342)
(200, 342)


In [14]:
print(type(vocab))
print(vocab)

<class 'set'>
{'certainly', 'shakespeare', 'heather', 'tim', 'peaks', 'surprise', 'brought', 'menace', 'needs', 'direct', 'look', 'seems', 'investigate', 'ludicrous', 'kids', 'pages', 'violence', 'crowd', 'keeping', 'casper', 'acting', 'limit', 'saying', 'briefed', 'theres', 'world', 'british', 'turning', 'killer', 'carrot', 'another', 'log', 'really', 'london', 'identity', 'finger', 'theyre', 'crime', 'burton', 'much', 'film', 'block', 'interesting', 'victorianera', 'attempt', 'horribly', 'hollow', 'locals', 'goulds', 'thats', 'deming', 'words', 'ripper', 'riddle', 'childs', 'les', 'comparison', 'supporting', 'series', 'get', 'starting', 'whistling', 'helped', 'first', 'like', 'new', 'arriving', 'half', 'onto', 'brothers', 'nearly', 'killing', 'prague', 'sexuality', 'campbell', 'jack', 'scenes', 'find', 'street', 'sooty', 'londons', 'nervous', 'language', 'crack', 'upon', 'mis', 'indians', 'reasons', 'unfortunates', 'question', 'would', 'vertical', 'martin', 'never', 'blame', 'actuall

In [9]:
def predict_sentiment(review, vocab, tokenizer, model):
	tokens = clean_doc(review)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	# convert to line
	line = ' '.join(tokens)
	# encode
	encoded = tokenizer.texts_to_matrix([line], mode='freq')
	# prediction
	yhat = model.predict(encoded, verbose=0)
	return round(yhat[0,0])

In [10]:
# test positive text
text = 'Best movie ever!'
print(predict_sentiment(text, vocab, tokenizer, model))
# test negative text
text = 'This is a bad movie.'
print(predict_sentiment(text, vocab, tokenizer, model))

0.0
0.0


In [5]:
print(type(positive_lines))
print(positive_lines[:5])
print(positive_lines)


<class 'list'>
['nothing one first oscar funny peter one job oscar sense oscar could watch named campbell point make course oscar job get go oscar one content well another good end oscar well oscar one oscar great oscar oscar words oscar mouth word oscar isnt one would think would oscar could job oscar see campbell star love oscar campbell never acting campbell film would look film times never well funny original cant dont think identity ending great one better', 'secret even world new look funny find theres place world films dont theres time would might completely well one original time course could see see film funny upon see roles films well dont look films particulars never get like make could wasnt interesting nothing scenes almost get really film well go film back joe say isnt', 'thats film thats seems think end thats actually film find film set flashbacks actually flashbacks get sense would first film well nothing nothing top word nothing case dont see good good', 'good one film

In [6]:
tokenizer = Tokenizer()
# fit the tokenizer on the documents
docs = positive_lines + negative_lines
tokenizer.fit_on_texts(docs)

NameError: name 'Tokenizer' is not defined